# Token classification (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
# !apt install git-lfs

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!unzip -nq archive_pkl.zip

In [10]:
import pandas as pd
df = pd.read_pickle('./df_token_with_classes.pkl')


Drop some data

In [11]:
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [12]:
df.head(3)

,tokens,classes
0,"[Вологодская, Область, Грязовецкий, Район, Вох...","[1, 2, 3, 4, 7, 8, 8]"
1,"[Ярославская, Область, Ярославский, Район, Ле-...","[1, 2, 3, 4, 5, 6, 6, 9, 10]"
2,"[Ленинградская, Область, Кировский, Район, Син...","[1, 2, 3, 4, 7, 8, 8, 9, 10]"


In [7]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)
train_testvalid = dataset.train_test_split(test_size=0.33, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
raw_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']})


In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'classes'],
        num_rows: 101794
    })
    validation: Dataset({
        features: ['tokens', 'classes'],
        num_rows: 25069
    })
    test: Dataset({
        features: ['tokens', 'classes'],
        num_rows: 25069
    })
})

In [9]:
raw_datasets["train"][0]["tokens"]

['Челябинская', 'Область', 'Магнитогорск', 'Город', 'Акварельная', 'Улица']

In [10]:
raw_datasets["train"][0]["classes"]

[1, 2, 7, 8, 9, 10]

In [11]:
ner_feature = raw_datasets["train"].features["classes"]
ner_feature

Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)

In [12]:
# label_names = ner_feature.feature.names
label_names = [ 
'OTHER',
'REGION',
'REGION_TYPE',
'AREA',
'AREA_TYPE',
'TERRITORY',
'TERRITORY_TYPE',
'CITY',
'CITY_TYPE',
'STREET',
'STREET_TYPE'
]
label_names

['OTHER',
 'REGION',
 'REGION_TYPE',
 'AREA',
 'AREA_TYPE',
 'TERRITORY',
 'TERRITORY_TYPE',
 'CITY',
 'CITY_TYPE',
 'STREET',
 'STREET_TYPE']

In [13]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["classes"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Челябинская Область     Магнитогорск Город     Акварельная Улица       
REGION      REGION_TYPE CITY         CITY_TYPE STREET      STREET_TYPE 


In [14]:
from transformers import AutoTokenizer

# model_checkpoint = "bert-base-cased"
model_checkpoint = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [15]:
tokenizer.is_fast

True

In [16]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Челябинска',
 '##я',
 'Область',
 'Магнитого',
 '##рск',
 'Город',
 'Ак',
 '##вар',
 '##ельная',
 'Улица',
 '[SEP]']

In [17]:
inputs.word_ids()

[None, 0, 0, 1, 2, 2, 3, 4, 4, 4, 5, None]

In [18]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            new_labels.append(label)

    return new_labels

In [19]:
labels = raw_datasets["train"][0]["classes"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[1, 2, 7, 8, 9, 10]
[-100, 1, 1, 2, 7, 7, 8, 9, 9, 9, 10, -100]


In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["classes"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [21]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/101794 [00:00<?, ? examples/s]

Map:   0%|          | 0/25069 [00:00<?, ? examples/s]

Map:   0%|          | 0/25069 [00:00<?, ? examples/s]

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [23]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(3)])
batch["labels"]

tensor([[-100,    1,    1,    2,    7,    7,    8,    9,    9,    9,   10, -100,
         -100, -100, -100, -100, -100, -100, -100, -100],
        [-100,    1,    1,    2,    3,    3,    4,    7,    7,    8,    5,    5,
            6,    6,    9,    9,    9,    9,   10, -100],
        [-100,    1,    2,    3,    3,    3,    4,    7,    7,    7,    8,    8,
            9,    9,    9,   10, -100, -100, -100, -100]])

In [24]:
for i in range(3):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 1, 1, 2, 7, 7, 8, 9, 9, 9, 10, -100]
[-100, 1, 1, 2, 3, 3, 4, 7, 7, 8, 5, 5, 6, 6, 9, 9, 9, 9, 10, -100]
[-100, 1, 2, 3, 3, 3, 4, 7, 7, 7, 8, 8, 9, 9, 9, 10, -100]


In [25]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
import evaluate

metric = evaluate.load("seqeval")

In [27]:
labels = raw_datasets["train"][0]["classes"]
labels = [label_names[i] for i in labels]
labels

['REGION', 'REGION_TYPE', 'CITY', 'CITY_TYPE', 'STREET', 'STREET_TYPE']

In [28]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'EGION': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'EGION_TYPE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ITY': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ITY_TYPE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'TREET': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'TREET_TYPE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.8333333333333334,
 'overall_f1': 0.9090909090909091,
 'overall_accuracy': 0.8333333333333334}

In [29]:
import numpy as np



def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [30]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [31]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
model.config.num_labels

11

In [33]:
#from huggingface_hub import notebook_login

#notebook_login()

In [34]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("rubert-address-elements")

In [36]:
mymodel = AutoModelForTokenClassification.from_pretrained("rubert-address-elements")

In [51]:
from transformers import pipeline

# Replace this with your own checkpoint
# model_checkpoint = "rubert-address-elements"
token_classifier = pipeline(
    task = "token-classification",
    model=mymodel,
    tokenizer=tokenizer,
    grouped_entities=True,
    ignore_subwords=True
)
res = token_classifier("Калужская Область Грайвороновский Район, Село Пищево Улица Земельная")
# res = token_classifier("Новосибирская Кировская, Область, Ново Лукский город")
res

[{'entity_group': 'REGION',
  'score': 1.0,
  'word': 'Калужская',
  'start': 0,
  'end': 9},
 {'entity_group': 'REGION_TYPE',
  'score': 0.9999999,
  'word': 'Область',
  'start': 10,
  'end': 17},
 {'entity_group': 'AREA',
  'score': 0.9999995,
  'word': 'Грайвороновский',
  'start': 18,
  'end': 33},
 {'entity_group': 'AREA_TYPE',
  'score': 0.9999995,
  'word': 'Район',
  'start': 34,
  'end': 39},
 {'entity_group': 'CITY',
  'score': 0.9999931,
  'word': ',',
  'start': 39,
  'end': 40},
 {'entity_group': 'CITY_TYPE',
  'score': 0.99998236,
  'word': 'Село',
  'start': 41,
  'end': 45},
 {'entity_group': 'STREET',
  'score': 0.9999682,
  'word': 'Пищево',
  'start': 46,
  'end': 52},
 {'entity_group': 'STREET_TYPE',
  'score': 0.99887925,
  'word': 'Улица Земельная',
  'start': 53,
  'end': 68}]